In [458]:
import pandas as pd
import numpy as np

#VARIABLES AND DATA
candles = pd.read_csv("XBTUSD-1m-data.csv")
candle = pd.Series()
    
def candleseries(candleamount):
    # iterate over rows with iterrows()
    for index, data in candles.head(candleamount).iterrows():
        #determine if candles are of opposite type
        if data['open'] < data['close']:
            type = "green"
        elif data['open'] > data['close']:
            type = "red"
        else:
            type = "abs_doji"
     #append size
        size = [data['open'], data['close'], abs(data['open']-data['close']), type]
        candle.at[index] = size

#realtime func
def tradesignal(i = -1, candleamount = 500, threshold = 1, ignoredoji = False):
    if candle.iloc[i][2] == candle.iloc[i-1][2]:
        return("WAIT")
    elif (candle.iloc[i][2] * threshold) > (candle.iloc[i-1][2]) and (ignoredoji == False or candle.iloc[i-1][2] > 1):
        if candle.iloc[i][3] == "red":
            return("SELL")
        elif candle.iloc[i][3] == "green":
            return("BUY")
        else: return("WAIT")
    else:
        return("WAIT")

#back-test on the series extrapolated from price data
def backtest(candleamount, threshold=1, ignoredoji=False):
    #first generate a candle-series!
    candleseries(candleamount)
    #generate index series for all candles except for the oldest one (no prev candle for it!)
    cseries = iter(candle.index[::-1])
    next(cseries)
    #list to store signals in for easier display
    signals=[]
    #generate a trade signal for every candle except for the last, and store in the list we created
    for i in cseries:
        i = -1*(i)
        signals.append(tradesignal(i, candleamount, threshold, ignoredoji))
    return(signals)

#run the back-test function
#store into data-frame to compare w/ different engulf parameters
btdata = pd.DataFrame()
btdata["T=.1,DOJI"] = backtest(50, .1)
btdata["T=.25,DOJI"] = backtest(50, .25)
btdata["T=.5,DOJI"] = backtest(50, .5)
btdata["T=1,DOJI"] = backtest(50, 1)
btdata["T=.1,nDOJI"] = backtest(50, .1)
btdata["T=.25,nDOJI"] = backtest(50, .25)
btdata["T=.5,nDOJI"] = backtest(50, .5)
btdata["T=1,DOJI"] = backtest(50, 1)
print(btdata)

    T=.1 T=.25  T=.5   T=1 T=1.1 T=1.25 T=1.5 T=1.75   T=2
0   WAIT  WAIT  WAIT  WAIT  WAIT   WAIT  WAIT   WAIT  WAIT
1   WAIT  WAIT  WAIT  WAIT  WAIT   WAIT  WAIT   WAIT  WAIT
2   WAIT  WAIT  WAIT  WAIT  WAIT   WAIT  WAIT   WAIT  WAIT
3   WAIT  WAIT  WAIT  WAIT  WAIT   WAIT  WAIT   WAIT  WAIT
4   SELL  SELL  SELL  SELL  SELL   SELL  SELL   SELL  SELL
5   SELL  SELL  SELL  SELL  SELL   SELL  SELL   SELL  SELL
6   WAIT  WAIT  WAIT  WAIT  WAIT   WAIT  WAIT   WAIT  WAIT
7   WAIT  WAIT  WAIT   BUY   BUY    BUY   BUY    BUY   BUY
8   WAIT  WAIT  WAIT  WAIT  WAIT   WAIT  WAIT   WAIT  WAIT
9   SELL  SELL  SELL  SELL  SELL   SELL  SELL   SELL  SELL
10  WAIT  WAIT  WAIT  WAIT  WAIT   WAIT  WAIT   WAIT  WAIT
11  WAIT  WAIT  WAIT  WAIT  WAIT   WAIT  WAIT   WAIT  WAIT
12  WAIT  WAIT  WAIT  WAIT  WAIT   WAIT  WAIT   WAIT  WAIT
13  WAIT  WAIT  WAIT  WAIT  WAIT   WAIT  WAIT   WAIT  WAIT
14   BUY   BUY   BUY   BUY   BUY    BUY   BUY    BUY   BUY
15  SELL  SELL  SELL  SELL  SELL   SELL  SELL   SELL  SE

C:\Users\eyese\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
